### **MiraMon Integral Index of Connectivity (ICT)** - calculating habitat connectivity

**NOTE**: for the command line steps please use your terminal app instead of running Jupyter cells as it allows to see full logs.


**1st step.** Our **GraphabProc** component compresses all output raster with decimal values using ZSTD compression. However, MiraMon ICT doesn't support this compression option. Therefore, we need to rewrite this data with `NONE` compression:


In [ ]:
for /R %s in (*.tif *.tiff) do (gdal_translate -of GTiff -co "COMPRESS=None" "%s" "%~dpns_not.tif" && del "%s" && ren "%~dpns_not.tif" "%~nxs")

The command above applies to all GeoTIFF datasets in the current directory, without definition.

**2nd step.** Then, we have to convert the original GeoTIFF datasets to the IMG format as MiraMon ICT supports only this format for input data. This will also create auxiliary `.rel` files, which are used by MiraMon to store metadata.

In [ ]:
for /R %s in (*.tif *.tiff) do @echo Processing: "%s" & D:\Programs\Miramon\TIFIMG 2 "%s" "%~dpns.img"

You will see many log entries, describing the intermediate steps.

**3rd step.** To launch ICT computations, run the prepared batch file. This batch file:
- founds matching filenames of input datasets (landscape impedance and landscape affinity)
- defines the filename of output connectivity dataset
- defines the parameters of MiraMon ICT command

List your case studies, when you run this batch file. In our example, these are `albera_buf_edge_upd` and `albera_buf_nedge_upd`.

For each case study you can define the parameters of sampling distance and exploration radius separately:
`case_study`:`distance`:`radius`, for example:

`albera_buf_edge_upd:30:525`.

For the input datasets with a spatial resolution of 390 m, covering the whole Catalonia, we used the parameters 390:585.
For the input datasets with a spatial resolution of 30 m, we used the parameters 30:525.

**IMPORTANT:** exploration radius, when doubled and divided by the cell side (spatial resolution) of the original files, should be equal to an odd number. Example: (525*2)/30 = 35 (valid).

**OTHER PARAMETERS**: 
1. You must also specify if the MiraMon ICT should rewrite existing output datasets if they've been created previously during your calculations:<br>
`auto_confirm`:`false` or `auto_confirm`:`true`<br><br>
2. It is mandatory to define your local directory where MiraMon ICT was installed:<br>
`miramon_dir`:`D:\Programs\Miramon`

In [ ]:
!task.bat albera_buf_edge_upd:30:525 albera_buf_nedge_upd:30:525 auto_confirm:true miramon_dir:D:\Programs\Miramon

^C


In the example above, we specify two case studies with the numerical parameters and also check if we would like to auto-confirm overwriting output datasets.


If everything is correct, you will see MiraMon ICT, calculating the index column by column, row by row in the same or separated window (depending on your terminal app). Please allow ample time to complete estimations. If you have previously calculated these indices, the program will ask if you want to overwrite the output files.

Once it is completed, let's check metadata of an of the Miramon ICT output files:

In [ ]:
from utils import check_metadata

check_metadata()

**4th step.** Once Miramon completed the computation, we need to transform the outputs in IMG format back to TIFF for subsequent analysis:

In [ ]:
for /R %s in (*.img) do @echo Processing: "%s" & D:\Programs\Miramon\TIFIMG 6 "%s" "%~dpns.tif" 

That's done, now you have Index of Terrestrial Connectivity outputs located in `data\case_study\ict` directory.